# Set-up

In [80]:
import os
import sys
import yaml
import logging
import mudata
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from typing import List, Dict, Tuple, Union, Optional, Literal

# Change path to wherever you have repo locally
sys.path.append('/cellar/users/aklie/opt/gene_program_evaluation')

from src.evaluation import (
    compute_categorical_association,
    compute_geneset_enrichment,
    compute_trait_enrichment,
    compute_perturbation_association,
    compute_explained_variance_ratio,
    compute_motif_enrichment
)
from src.evaluation.enrichment_trait import process_enrichment_data

In [84]:
# I/O paths
path_config = "/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF_30/evaluation_pipeline.yml"
config = yaml.safe_load(open(path_config))

## I/O

In [85]:
io_config = config['io']
io_config

{'path_mdata': '/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/cNMF/cNMF_30_0.2_gene_names.h5mu',
 'path_out': '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF_30',
 'data_key': 'rna',
 'prog_key': 'cNMF'}

In [8]:
# Load mdata
path_mdata = io_config['path_mdata']
mdata = mudata.read(path_mdata)
mdata

/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


MuData object with n_obs × n_vars = 92284 × 5481
  2 modalities
    rna:	92284 x 5451
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
      obsm:	'X_pca', 'X_umap'
      layers:	'norm10k'
    cNMF:	92284 x 30
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
      uns:	'guide_names', 'guide_targets', 'var_names'
      obsm:	'guide_assignment'
      varm:	'loadings'

In [9]:
prog_key = "cNMF"
data_key = "rna"

In [45]:
# choose the first 3 programs in prog_key
prog_names = list(mdata.mod[prog_key].var_names)[:3]
mdata.mod[prog_key] = mdata.mod[prog_key][:, prog_names]
mdata

MuData object with n_obs × n_vars = 92284 × 5481
  2 modalities
    rna:	92284 x 5451
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
      obsm:	'X_pca', 'X_umap'
      layers:	'norm10k'
    cNMF:	92284 x 3
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
      uns:	'guide_names', 'guide_targets', 'var_names'
      obsm:	'guide_assignment'
      varm:	'loadings'

# Gene set enrichment testing

In [86]:
from src.evaluation.enrichment_geneset import get_geneset, get_program_gene_loadings
import gseapy as gp

In [87]:
gene_set_enrichment_config = config['gene_set_enrichment']
gene_set_enrichment_config

{'prog_nam': None,
 'organism': 'human',
 'libraries': ['Reactome_2022', 'GO_Biological_Process_2023'],
 'method': 'fisher',
 'database': 'enrichr',
 'n_top': 500,
 'low_cutoff': 0,
 'n_jobs': -1,
 'inplace': False,
 'user_geneset': None,
 'max_size': 500,
 'min_size': 5}

## Get geneset and loadings

In [66]:
reactome = get_geneset(
    organism="human",
    library="Reactome_2022",
    database="enrichr",
    max_size=500,
    min_size=5
)

INFO:root:Downloading and generating Enrichr library gene sets...
INFO:root:Library is already downloaded in: /cellar/users/aklie/.cache/gseapy/Enrichr.Reactome_2022.gmt, use local file
INFO:root:0031 gene_sets have been filtered out when max_size=500 and min_size=5


In [67]:
loadings = get_program_gene_loadings(
    mdata, 
    prog_key=prog_key, 
    prog_nam=gene_set_enrichment_config['prog_nam'],
    data_key=data_key, 
    organism=gene_set_enrichment_config['organism'],
)

## GSEA

In [73]:
def perform_prerank(
    loadings: pd.DataFrame,
    geneset: Union[List[str], str, Dict[str, str]],
    n_jobs: int = 1,
    low_cutoff: float = -np.inf,
    n_top: Optional[int] = None,
    **kwargs
) -> pd.DataFrame:
    """Run GSEA prerank on each gene program in the loadings matrix.
    
    Parameters
    ----------
    loadings : pd.DataFrame
        DataFrame of feature loadings for each program.
    geneset : str
        Name of the set to run GSEA on.
    n_jobs : int
        Number of parallel jobs to run.
    low_cutoff : float
        Remove features with loadings at or below this value.
    n_top : int
        Take the top n features with the highest loadings.
        
    Returns
    -------
    pd.DataFrame
        DataFrame of GSEA results sorted by program name and FDR q-value. Includes the following columns:
        - program_name: name of the program
        - Term: gene set name
        - ES: enrichment score
        - NES: normalized enrichment score
        - NOM p-val: nominal p-value (from the null distribution of the gene set)
        - FDR q-val: adjusted False Discovery Rate
        - FWER p-val: Family wise error rate p-values
        - Gene %: percent of gene set before running enrichment peak (ES)
        - Lead_genes: leading edge genes (gene hits before running enrichment peak)
        - tag_before: number of genes in gene set
        - tag_after: number of genes matched to the data
    """

    # Run GSEA prerank for each column of loadings (each cell program)
    pre_res = pd.DataFrame()
    for i in tqdm(loadings.columns, desc='Running GSEA', unit='programs'):

        # Filter out low loadings
        temp_loadings = loadings[i][(loadings[i] > low_cutoff)]

        # Take top n features if specified
        if n_top is not None:
            temp_loadings = temp_loadings.sort_values(ascending=False).head(n_top)
            if len(temp_loadings) < n_top:
                logging.warning(f"Program {i} has less than {n_top} features after filtering. Only {len(temp_loadings)} features will be used.")

        # Run GSEA prerank
        temp_res = gp.prerank(
            rnk=temp_loadings, 
            gene_sets=geneset, 
            threads=n_jobs, 
            **kwargs
        ).res2d

        # Post-process results
        temp_res['Gene %'] = temp_res['Gene %'].apply(lambda x: float(x[:-1]))
        temp_res['tag_before'] = temp_res['Tag %'].apply(lambda x: int(x.split('/')[0]))
        temp_res['tag_after'] = temp_res['Tag %'].apply(lambda x: int(x.split('/')[1]))
        temp_res.drop(columns=['Tag %'], inplace=True)
        if 'Name' in temp_res.columns and temp_res['Name'][0] == "prerank":
            temp_res['Name'] = i
        temp_res.rename(columns={'Name': 'program_name'}, inplace=True)
        temp_res = temp_res.sort_values(['program_name', 'FDR q-val'])
        pre_res = pd.concat([pre_res, temp_res], ignore_index=True)
    
    return pre_res

In [74]:
pre_res_gsea = perform_prerank(
    loadings=loadings, 
    geneset=reactome,
    n_jobs=gene_set_enrichment_config['n_jobs'],
    low_cutoff=0,
    n_top=None
)

Running GSEA:   0%|          | 0/3 [00:00<?, ?programs/s]

2024-09-07 12:33:51,783 [WARNING] Duplicated values found in preranked stats: 0.03% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


In [75]:
pre_res_gsea_500 = perform_prerank(
    loadings=loadings, 
    geneset=reactome,
    n_jobs=gene_set_enrichment_config['n_jobs'],
    low_cutoff=0,
    n_top=500
)

Running GSEA:   0%|          | 0/3 [00:00<?, ?programs/s]

In [99]:
def perform_fisher_enrich(
    loadings, 
    geneset, 
    n_top=500,
    **kwargs
):
    """Run Fisher enrichment on each gene program in the loadings matrix.

    Parameters
    ----------
    loadings : pd.DataFrame
        DataFrame of feature loadings for each program.
    geneset : dict
        Dictionary of gene sets.
    n_top : int
        Number of top features to take.
    
    Returns
    -------
    ['Gene_set', 'Term', 'P-value', 'Adjusted P-value', 'Odds Ratio',
       'Combined Score', 'Genes', 'program_name', 'overlap_numerator',
       'overlap_denominator'],
    pd.DataFrame
        DataFrame of Fisher enrichment results sorted by program name and adjusted p-value. Includes the following columns:
        - program_name: name of the program
        - Term: gene set name
        - P-value: Fisher's exact test p-value
        - Adjusted P-value: adjusted p-value
        - Odds Ratio: odds ratio
        - Combined Score: combined score
        - Genes: genes in the gene set
        - overlap_numerator: number of overlapping genes
        - overlap_denominator: number of genes in the gene set
    TODO
    ----
    - Parallelize across programs
    """

    # Find the intersection of genes present in the mudata object and in the library
    background_genes = set(value for sublist in geneset.values() for value in sublist)
    
    enr_res = pd.DataFrame()
    for i in tqdm(loadings.columns, desc='Running Fisher enrichment', unit='programs'):
        gene_list = list(loadings[i].sort_values(ascending=False).head(n_top).index)
        temp_res = gp.enrich(
            gene_list=gene_list,
            gene_sets=geneset, 
            background=background_genes
        ).res2d
        temp_res["program_name"] = i
        enr_res = pd.concat([enr_res, temp_res], ignore_index=True)
    enr_res['overlap_numerator'] = enr_res['Overlap'].apply(lambda x: int(x.split('/')[0]))
    enr_res['overlap_denominator'] = enr_res['Overlap'].apply(lambda x: int(x.split('/')[1]))
    enr_res.drop(columns=['Overlap', 'Gene_set'], inplace=True)
    enr_res = enr_res[["program_name"] + [col for col in enr_res.columns if col != "program_name"]]
    enr_res = enr_res.sort_values(['program_name', 'Adjusted P-value'])
    
    return enr_res

In [100]:
pre_res_fisher_500 = perform_fisher_enrich(
    loadings=loadings,
    geneset=reactome,
    n_top=500
)

Running Fisher enrichment:   0%|          | 0/3 [00:00<?, ?programs/s]

## Wrapper

In [88]:
def insert_enrichment(
    mdata: mudata.MuData,
    df: pd.DataFrame,
    library="GSEA", 
    prog_key="prog",
    geneset_index="Term", 
    program_index="program_name",
    varmap_name_prefix="gsea_varmap"
) -> None:
    """Insert geneset enrichment into mudata
    
    Parameters
    ----------
    mdata : mudata.MuData
        MuData object.
    df : pd.DataFrame
        DataFrame of geneset enrichment results.
    library : str
        name of the library used for enrichment.
    prog_key : str
        key for the program in the mdata object.
    geneset_index : str
        index for the gene set in the DataFrame.
    program_index : str
        index for the program in the DataFrame.
    varmap_name_prefix : str
        prefix for the varmap name.
    
    Returns
    -------
    None
    """
    
    # Create a mudata key to column name mapping dictionary
    mudata_keys_dict = {}
    for col in df.columns:
        if col not in [geneset_index, program_index]:
            key = f"{col}_{library}"
            key = key.replace(' ', '_').replace('%', 'percent')
            mudata_keys_dict[key] = col

    # Insert the values from the dataframe into the array for each key
    for key, colname in mudata_keys_dict.items():
        # Create an empty dataframe with the right dimensions
        all_progs_df = pd.DataFrame(index=df[geneset_index].unique(), 
                                    columns=mdata[prog_key].var.index)
        
        # Pivot the dataframe for gene sets and programs
        pivot_df = df[[geneset_index, program_index, colname]].pivot(index=geneset_index, 
                                                                     columns=program_index, 
                                                                     values=colname)
        
        # Update the empty dataframe with new values
        all_progs_df[pivot_df.columns] = pivot_df
        
        # Convert dataframe to a numpy array
        all_progs_array = all_progs_df.T.to_numpy()
        
        # Add the array into the MuData object
        mdata[prog_key].varm[key] = all_progs_array
        
    # Add the varmap to the mudata object
    varmap_name = f"{varmap_name_prefix}_{library}"
    mdata[prog_key].uns[varmap_name] = all_progs_df.index


def compute_geneset_enrichment(
    mdata: Union[str, mudata.MuData],
    prog_key: str = 'program_name',
    data_key: str = 'rna',
    prog_name: Optional[str] = None,
    method: Literal['gsea', 'fisher'] = 'gsea',
    organism: Literal['human', 'mouse'] = 'human',
    library: str = 'Reactome_2022', 
    database: Literal['msigdb', 'enrichr'] = 'enrichr',
    user_geneset: Optional[Dict[str, List[str]]] = None,
    min_size: int = 0,
    max_size: int = 2000,
    low_cutoff: float = -np.inf,
    n_top: Optional[int] = None,
    n_jobs: int = 1,
    inplace: bool = True,
    **kwargs
) -> Optional[pd.DataFrame]:

    """
    Wrapper function to compute gene set enrichment for each program in the MuData object.

    Parameters
    ----------
    mdata : Union[str, mudata.MuData]
        Path to the MuData object or the MuData object itself.
    prog_key : str
        index for the anndata object (mdata[prog_key]) in the mudata object.
    data_key : str
        index of the genomic data anndata object (mdata[data_key]) in the mudata object.
    prog_name : str (default: None)
        Compute enrichment for a particular program.
    method : {'gsea', 'fisher'} (default: 'gsea')
        Run GSEA or Fisher exact test gene set enrichment.
    organism : {'human', 'mouse'} (default: 'human')
        species to which the sequencing data was aligned to.
    library : str (default: Reactome_2022)
        gene-set library to use for computing enrichment.
        MsigDB libraries: https://www.gsea-msigdb.org/gsea/msigdb
        Enrichr libraries: https://maayanlab.cloud/Enrichr/#libraries
    database : {'msigdb', 'enrichr'} (default: 'enrichr')
        database of gene-set libraries to use. Should match the library.
    user_geneset : dict
        user-defined gene set to use for enrichment. If provided, library and database are ignored.
    min_size: int (default: 0)
        minimum size of gene sets to consider.
    max_size: int (default: 2000)
        maximum size of gene sets to consider.
    low_cutoff : float (default: -np.inf)
        Remove features with loadings at or below this value.
    n_top : int
        Take the top n features with the highest loadings.
    n_jobs : int (default: 1)
        number of threads to run processes on.
    inplace : bool (default: True)
        whether to insert the results back into the mudata object.
    
    Returns
    -------
    if not inplace:
        return pre_res
    else:
        inserts the results back into the mudata object
    """

    # Read in mudata if it is provided as a path
    frompath=False
    if isinstance(mdata, str):
        if os.path.exists(mdata):
            mdata = mudata.read(mdata)
            if inplace:
                logging.warning('Changed to inplace=False since path was provided')
                inplace=False
            frompath=True
        else: raise ValueError('Incorrect mudata specification.')
    
    # get the geneset
    if user_geneset is not None:
        geneset = user_geneset
    else:
        geneset = get_geneset(
            organism=organism,
            library=library,
            database=database,
            max_size=max_size,
            min_size=min_size
        )
     
    # get the gene loadings for each program
    loadings = get_program_gene_loadings(
        mdata, 
        prog_key=prog_key, 
        prog_nam=prog_name,
        data_key=data_key, 
        organism=organism,
    )
    
    # run enrichment
    if method == "gsea":
        pre_res = perform_prerank(
            loadings=loadings, 
            geneset=geneset,
            n_jobs=n_jobs,
            low_cutoff=low_cutoff,
            n_top=n_top,
            **kwargs
        )
    elif method == "fisher":
        pre_res = perform_fisher_enrich(
            loadings=loadings,
            geneset=geneset,
            n_top=n_top,
            **kwargs
        )
        
    # insert results back into the mudata object if inplace
    if inplace:
        mdata=insert_enrichment(
            mdata, df=pre_res, 
            library=library, 
            prog_key=prog_key,
            geneset_index="Term", 
            program_index="program_name",
            varmap_name_prefix="gsea_varmap"
        )
    else:
        return(pre_res)

In [90]:
# Run wrapper function with GSEA
pre_res_wrapper_gsea = compute_geneset_enrichment(
    mdata=mdata,
    prog_key=prog_key,
    data_key=data_key,
    method='gsea',
    organism=gene_set_enrichment_config['organism'],
    library='Reactome_2022',
    database=gene_set_enrichment_config['database'],
    user_geneset=gene_set_enrichment_config['user_geneset'],
    min_size=gene_set_enrichment_config['min_size'],
    max_size=gene_set_enrichment_config['max_size'],
    n_jobs=gene_set_enrichment_config['n_jobs'],
    low_cutoff=gene_set_enrichment_config['low_cutoff'],
    n_top=gene_set_enrichment_config['n_top'],
    inplace=False
)

INFO:root:Downloading and generating Enrichr library gene sets...
INFO:root:Library is already downloaded in: /cellar/users/aklie/.cache/gseapy/Enrichr.Reactome_2022.gmt, use local file
INFO:root:0002 gene_sets have been filtered out when max_size=2000 and min_size=0


Running GSEA:   0%|          | 0/3 [00:00<?, ?programs/s]

2024-09-07 12:51:59,556 [WARNING] Duplicated values found in preranked stats: 0.03% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


In [93]:
pre_res_wrapper_gsea.head()

,program_name,Term,ES,NES,NOM p-val,FDR q-val,FWER p-val,Gene %,Lead_genes,tag_before,tag_after
0,0,TCF Dependent Signaling In Response To WNT R-H...,0.752635,1.53758,0.0,0.122341,0.095,11.69,SFRP1;TCF7L1;SOX2;SFRP2;TCF7L2;SOX4;TLE1;HECW1...,11,28
1,0,Glucagon-like Peptide-1 (GLP1) Regulates Insul...,0.731538,1.415272,0.013065,0.137839,0.658,11.83,ITPR2;GNG4;PRKCA;GNGT1;ADCY5;ITPR3;PRKAR1B,7,15
2,0,Opioid Signaling R-HSA-111885,0.698056,1.403416,0.005005,0.142552,0.713,19.67,ADCY2;ITPR2;NBEA;GNG4;PRKCA;GNGT1;ADCY5;ITPR3;...,13,25
3,0,Signaling By WNT R-HSA-195721,0.712681,1.501559,0.0,0.142899,0.202,11.69,SFRP1;TCF7L1;ITPR2;SOX2;SFRP2;TCF7L2;FZD7;GNG4...,21,50
4,0,Semaphorin Interactions R-HSA-373755,0.71259,1.4068,0.013039,0.145824,0.699,15.36,DPYSL3;SEMA6A;SEMA3A;ERBB2;SEMA4D;SEMA5A;CRMP1,7,16


In [97]:
# Run wrapper function with Fisher
pre_res_wrapper_fisher = compute_geneset_enrichment(
    mdata=mdata,
    prog_key=prog_key,
    data_key=data_key,
    method='fisher',
    organism=gene_set_enrichment_config['organism'],
    library='Reactome_2022',
    database=gene_set_enrichment_config['database'],
    user_geneset=gene_set_enrichment_config['user_geneset'],
    min_size=gene_set_enrichment_config['min_size'],
    max_size=gene_set_enrichment_config['max_size'],
    n_jobs=gene_set_enrichment_config['n_jobs'],
    low_cutoff=gene_set_enrichment_config['low_cutoff'],
    n_top=gene_set_enrichment_config['n_top'],
    inplace=False
)

INFO:root:Downloading and generating Enrichr library gene sets...
INFO:root:Library is already downloaded in: /cellar/users/aklie/.cache/gseapy/Enrichr.Reactome_2022.gmt, use local file
INFO:root:0031 gene_sets have been filtered out when max_size=500 and min_size=5


Running Fisher enrichment:   0%|          | 0/3 [00:00<?, ?programs/s]

In [98]:
pre_res_wrapper_fisher.head()

,Term,P-value,Adjusted P-value,Odds Ratio,Combined Score,Genes,program_name,overlap_numerator,overlap_denominator
0,ADORA2B Mediated Anti-Inflammatory Cytokine Pr...,0.058902,0.323762,2.243373,6.352975,GPR176;GNG4;ADCY5;PRKCA;ADCY2;GNGT1;PRKAR1B,0,7,131
1,ADP Signaling Thru P2Y Purinoceptor 1 R-HSA-41...,0.027401,0.195146,5.765478,20.739340,GNGT1;GNA14;GNG4,0,3,25
2,ADP Signaling Thru P2Y Purinoceptor 12 R-HSA-3...,0.113993,0.476330,4.503989,9.780925,GNGT1;GNG4,0,2,22
3,AKT Phosphorylates Targets In Nucleus R-HSA-19...,0.235274,0.596152,5.816231,8.416101,FOXO1,0,1,10
4,ALK Mutants Bind TKIs R-HSA-9700645,0.275242,0.638869,4.803741,6.197321,BCL11A,0,1,12


# DONE!

---